In [1]:
from __future__ import print_function
import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.optimizers import RMSprop
import os
from time import time
import datetime
from keras.callbacks import TensorBoard
import tensorflow as tf
from keras import backend as K
import multiprocessing as mp


Using TensorFlow backend.


In [5]:
num_CPU=mp.cpu_count()

config = tf.ConfigProto(intra_op_parallelism_threads=8,\
        inter_op_parallelism_threads=8, allow_soft_placement=True,\
        device_count = {'CPU' : num_CPU, 'GPU' : 0})
session = tf.Session(config=config)
K.set_session(session)

In [2]:
tensorboard = TensorBoard(log_dir="logs/{}".format(time()))

In [3]:
batch_size = 32
num_classes = 10
epochs = 100
data_augmentation = True
num_predictions = 20
save_dir = os.path.join(os.getcwd(), 'saved_models')
model_name = 'keras_cifar10_trained_model.h5'

# The data, split between train and test sets:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# Convert class vectors to binary class matrices.
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)


x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


In [10]:
model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=x_train.shape[1:]))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

# initiate RMSprop optimizer
opt = RMSprop(lr=0.0001, decay=1e-6)

# Let's train the model using RMSprop
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

In [11]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

# CPU Trial

In [13]:
start = time()

if not data_augmentation:
    print('Not using data augmentation.')
    model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(x_test, y_test),
              shuffle=True, callbacks=[tensorboard])
else:
    print('Using real-time data augmentation.')
    # This will do preprocessing and realtime data augmentation:
    datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        zca_epsilon=1e-06,  # epsilon for ZCA whitening
        rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
        # randomly shift images horizontally (fraction of total width)
        width_shift_range=0.1,
        # randomly shift images vertically (fraction of total height)
        height_shift_range=0.1,
        shear_range=0.,  # set range for random shear
        zoom_range=0.,  # set range for random zoom
        channel_shift_range=0.,  # set range for random channel shifts
        # set mode for filling points outside the input boundaries
        fill_mode='nearest',
        cval=0.,  # value used for fill_mode = "constant"
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False,  # randomly flip images
        # set rescaling factor (applied before any other transformation)
        rescale=None,
        # set function that will be applied on each input
        preprocessing_function=None,
        # image data format, either "channels_first" or "channels_last"
        data_format=None,
        # fraction of images reserved for validation (strictly between 0 and 1)
        validation_split=0.0)

    # Compute quantities required for feature-wise normalization
    # (std, mean, and principal components if ZCA whitening is applied).
    datagen.fit(x_train)

    # Fit the model on the batches generated by datagen.flow().
    model.fit_generator(datagen.flow(x_train, y_train,
                                     batch_size=batch_size),
                        epochs=epochs,
                        validation_data=(x_test, y_test),
                        workers=4,
                        steps_per_epoch=len(x_train)/batch_size, 
                        callbacks=[tensorboard])

Using real-time data augmentation.
Epoch 1/100
1563/1562 [==============================] - 108s 69ms/step - loss: 1.4040 - acc: 0.4948 - val_loss: 1.2460 - val_acc: 0.5517
Epoch 2/100
1563/1562 [==============================] - 108s 69ms/step - loss: 1.3294 - acc: 0.5273 - val_loss: 1.1895 - val_acc: 0.5760
Epoch 3/100
1563/1562 [==============================] - 108s 69ms/step - loss: 1.2715 - acc: 0.5462 - val_loss: 1.1395 - val_acc: 0.5928
Epoch 4/100
1563/1562 [==============================] - 108s 69ms/step - loss: 1.2205 - acc: 0.5685 - val_loss: 1.0972 - val_acc: 0.6142
Epoch 5/100
1563/1562 [==============================] - 107s 69ms/step - loss: 1.1812 - acc: 0.5821 - val_loss: 1.0276 - val_acc: 0.6361
Epoch 6/100
1563/1562 [==============================] - 107s 69ms/step - loss: 1.1433 - acc: 0.5968 - val_loss: 1.0328 - val_acc: 0.6341
Epoch 7/100
1563/1562 [==============================] - 107s 69ms/step - loss: 1.1157 - acc: 0.6033 - val_loss: 0.9587 - val_acc: 0.6601

Epoch 60/100
1563/1562 [==============================] - 107s 69ms/step - loss: 0.7791 - acc: 0.7375 - val_loss: 0.7208 - val_acc: 0.7582
Epoch 61/100
1563/1562 [==============================] - 108s 69ms/step - loss: 0.7773 - acc: 0.7364 - val_loss: 0.7074 - val_acc: 0.7626
Epoch 62/100
1563/1562 [==============================] - 108s 69ms/step - loss: 0.7816 - acc: 0.7350 - val_loss: 0.7771 - val_acc: 0.7399
Epoch 63/100
1563/1562 [==============================] - 108s 69ms/step - loss: 0.7826 - acc: 0.7361 - val_loss: 0.7023 - val_acc: 0.7639
Epoch 64/100
1563/1562 [==============================] - 107s 69ms/step - loss: 0.7739 - acc: 0.7414 - val_loss: 0.6700 - val_acc: 0.7798
Epoch 65/100
1563/1562 [==============================] - 108s 69ms/step - loss: 0.7790 - acc: 0.7375 - val_loss: 0.6878 - val_acc: 0.7688
Epoch 66/100
1563/1562 [==============================] - 108s 69ms/step - loss: 0.7803 - acc: 0.7368 - val_loss: 0.8417 - val_acc: 0.7386
Epoch 67/100
1563/1562 [===

In [15]:
# Save model and weights
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
model_path = os.path.join(save_dir, model_name)
model.save(model_path)
print('Saved trained model at %s ' % model_path)

# Score trained model.
scores = model.evaluate(x_test, y_test, verbose=1)

end = time()
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])
print('CPU Start:', str(start))
print('CPU End:', str(end))

Saved trained model at /home/ubuntu/code/DeepNeuralNetworksandAI/Assignment1/part2/saved_models/keras_cifar10_trained_model.h5 
10000/10000 [==============================] - 6s 553us/step
Test loss: 0.8414081791877747
Test accuracy: 0.7259
CPU Start: 00:00:00
CPU End: 00:00:00


In [16]:
time()

1548552426.282937

# GPU Trial

In [7]:
num_CPU=mp.cpu_count()

config = tf.ConfigProto(intra_op_parallelism_threads=8,\
        inter_op_parallelism_threads=8, allow_soft_placement=True,\
        device_count = {'CPU' : num_CPU, 'GPU' : 1})
session = tf.Session(config=config)
K.set_session(session)

K.tensorflow_backend._get_available_gpus()

[]

In [20]:
model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=x_train.shape[1:]))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

# initiate RMSprop optimizer
opt = RMSprop(lr=0.0001, decay=1e-6)

# Let's train the model using RMSprop
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

In [21]:
start = time()

if not data_augmentation:
    print('Not using data augmentation.')
    model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(x_test, y_test),
              shuffle=True, callbacks=[tensorboard])
else:
    print('Using real-time data augmentation.')
    # This will do preprocessing and realtime data augmentation:
    datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        zca_epsilon=1e-06,  # epsilon for ZCA whitening
        rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
        # randomly shift images horizontally (fraction of total width)
        width_shift_range=0.1,
        # randomly shift images vertically (fraction of total height)
        height_shift_range=0.1,
        shear_range=0.,  # set range for random shear
        zoom_range=0.,  # set range for random zoom
        channel_shift_range=0.,  # set range for random channel shifts
        # set mode for filling points outside the input boundaries
        fill_mode='nearest',
        cval=0.,  # value used for fill_mode = "constant"
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False,  # randomly flip images
        # set rescaling factor (applied before any other transformation)
        rescale=None,
        # set function that will be applied on each input
        preprocessing_function=None,
        # image data format, either "channels_first" or "channels_last"
        data_format=None,
        # fraction of images reserved for validation (strictly between 0 and 1)
        validation_split=0.0)

    # Compute quantities required for feature-wise normalization
    # (std, mean, and principal components if ZCA whitening is applied).
    datagen.fit(x_train)

    # Fit the model on the batches generated by datagen.flow().
    model.fit_generator(datagen.flow(x_train, y_train,
                                     batch_size=batch_size),
                        epochs=epochs,
                        validation_data=(x_test, y_test),
                        workers=4,
                        steps_per_epoch=len(x_train)/batch_size,
                        callbacks=[tensorboard])

Using real-time data augmentation.
Epoch 1/100
 249/1562 [===>..........................] - ETA: 1:26 - loss: 2.2036 - acc: 0.1756

KeyboardInterrupt: 

In [ ]:
model_name = 'gpu_keras_cifar10_trained_model.h5'

# Save model and weights
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
model_path = os.path.join(save_dir, model_name)
model.save(model_path)
print('Saved trained model at %s ' % model_path)

# Score trained model.
scores = model.evaluate(x_test, y_test, verbose=1)

end = time()
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])
print('GPU Runtime:', str(end-start))